<a href="https://colab.research.google.com/github/AryanMethil/Music-Generation-/blob/main/Melody_Generation_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import json
import numpy as np
import music21 as m21

In [ ]:
reloaded_model=tf.keras.models.load_model('/content/drive/My Drive/Melody Generation/my_model.h5')

In [ ]:
SEQUENCE_LENGTH=64
MAPPING_PATH='/content/drive/My Drive/Melody Generation/mapping.json'

In [ ]:
class MelodyGenerator():
  def __init__(self,model=reloaded_model):
    self.model=model

    with open(MAPPING_PATH,'r') as f:
      self.mappings=json.load(f)
    
    self.start_symbols=['/']*SEQUENCE_LENGTH
  
  def sample_with_temperature(self,probabilities,temperature):

    # temperature -> infinity [the probabilities dont matter and the symbol will be completely random]
    # temperature -> 0 [the highest probability one will always be chosen]
    # temperature -> 1 [the distribution of probabilities is used and so the highest prob one will be likely to get picked but wont necessarily always get picked]

    predictions = np.log(probabilities) / temperature
    probabilities = np.exp(predictions) / np.sum(np.exp(predictions))

    choices=range(len(probabilities))
    index=np.random.choice(choices,p=probabilities)

    return index

  def generate_melody(self,seed,num_steps,max_sequence_length,temperature):

    # create seed with start symbols
    seed=seed.split()
    melody=seed
    seed=self.start_symbols + seed

    # map seed list to int
    seed=[self.mappings[symbol] for symbol in seed]

    for _ in range(num_steps):

      # limit the seed to the last max_sequence_length symbols
      seed= seed[-max_sequence_length:]

      #one-hot encode
      onehot_seed=tf.keras.utils.to_categorical(seed,num_classes=len(self.mappings))

      # add an extra dimension representing a batch size
      onehot_seed=onehot_seed[np.newaxis,...]

      # predict
      probabilities=self.model.predict(onehot_seed)[0]

      output_int=self.sample_with_temperature(probabilities,temperature)

      # update the seed
      seed.append(output_int)

      output_symbol=[k for k,v in self.mappings.items() if v==output_int][0]

      # check whether we're at the end of a melody
      if(output_symbol=='/'):
        break
      
      # else update the melody
      melody.append(output_symbol)

    return melody
  
  def save_melody(self,melody,step_duration=0.25, format='midi',file_name='/content/drive/My Drive/Melody Generation/mel.midi'):

    # create a music21 stream
    stream = m21.stream.Stream()

    # parse all symbols ad create note/rest objects
    start_symbol=None
    step_counter=1

    for i,symbol in enumerate(melody):

      # handle the case where we have a note/rest
      if(symbol!='_' or i+1==len(melody)):
        
        # ensure that we are not dealing with the first event
        if(start_symbol is not None):

          quarter_length_duration=step_duration*step_counter

          #handle rest
          if(start_symbol=='r'):
            m21_event=m21.note.Rest(quarterLength=quarter_length_duration)

          #handle note
          else:
            m21_event=m21.note.Note(int(start_symbol),quarterLength=quarter_length_duration)

          stream.append(m21_event)

          # reset step counter
          step_counter=1

        start_symbol=symbol

      # handle the case in which we have '_'
      else:
        step_counter+=1

    # write the m21 stream to file
    stream.write(format,file_name)

    

In [ ]:
import os
import zipfile

local_zip = '/content/drive/My Drive/Melody Generation/dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/')

zip_ref.close()

In [ ]:
seed='60 _ 62 _ 64 _ _ _ 64 _ _ _ 65 _ 62 _ 62 _ 64 _ 64 _ _ _ 64 _ 65 _ 67 _ _ _ 65 _ 64 _ 62 _ 60 _ 59 _ 62 _ 62 _ _ _ 62 _ 64 _ 65 _ _ _ _ _ 69 _ 67 _ 65 _ 64 _ _ _ r _ _ _ 67 _ 72 _ 69 _ _ _ _ _ _ _ 67 _ 65 _ 64 _ _ _ _ _ 65 _ 62 _ _ _ 60 _ _ _ '
mg=MelodyGenerator()
melody=mg.generate_melody(seed,500,SEQUENCE_LENGTH,1)
print(melody)
mg.save_melody(melody)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


['60', '_', '62', '_', '64', '_', '_', '_', '64', '_', '_', '_', '65', '_', '62', '_', '62', '_', '64', '_', '64', '_', '_', '_', '64', '_', '65', '_', '67', '_', '_', '_', '65', '_', '64', '_', '62', '_', '60', '_', '59', '_', '62', '_', '62', '_', '_', '_', '62', '_', '64', '_', '65', '_', '_', '_', '_', '_', '69', '_', '67', '_', '65', '_', '64', '_', '_', '_', 'r', '_', '_', '_', '67', '_', '72', '_', '69', '_', '_', '_', '_', '_', '_', '_', '67', '_', '65', '_', '64', '_', '_', '_', '_', '_', '65', '_', '62', '_', '_', '_', '60', '_', '_', '_', '_', '_', '79', '59', '62', '70', '76', '68', '76', '_', '_', '_', '76', '68', '_', '_', '70', '_', '70', '_', '79', '68', '70', '70', '76', '_', '_', '_', '76', '68', '70', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', '69', 